# Computational Methods in Simulation (CMIS) Week 5
This notebook is intended to help students make their hand-in in this week of CMIS.

Hand-ins must be submitted as a maximum 3 page pdf file in Absalon and must be made using the ACM TOG overleaf latex template

https://www.overleaf.com/latex/templates/association-for-computing-machinery-acm-large-2-column-format-template/qwcgpbmkkvpq

Notebooks must be uploaded as well so teachers have code solutions accessible if needed.

This week is based on Slides 19

* Make an implementation of the 2D elastic assembly process
* Discuss from a computer science viewpoint which steps are the same and which steps that are different in the 2D elastic case compared to the 2D and 1D Poisson problems done in week 4.
* Speculate how to write a general FEM computer program that could simulate any partial differential equation with point wise boundary conditions.
* For the 2D elasticity problem, 
$\int_{\Omega^e}\left(\delta \varepsilon\right)^T\sigma d\Omega = \int_{\Gamma^e}\left(\delta u \right)^T f dS $
where $f$ is a given constant load (force per meter) use the FEM method to derive the $K_e$ and $f_e$ terms such that $K_e\,u_e =f_e$
* Based on your FEM derivation implement a rectangular solid 2D bar of dimension 6-by-2 meters and made of steel like material. Attach the left side of the bar to a wall (use boundary condition $u = 0$) and apply a downward nodal load on the right edge of the bar.    
* Examine the eigenvalue spectrum of the global stiffness matrix before applying boundary conditions and after (Hint: write down how many zero eigenvalues you expect before and after applying boundary conditions and explain how the minimum eigenvalue is related to the boundary conditions after having applied these).
* Examine the fill pattern of the stiffness matrix (Hint: reflect about the shape of the fill-in and how the non-zero values in $K$ is related to the mesh).
* Investigate what happens to the deformations when you vary the resolution and load on the bar. Try to find the best suitable mesh resolution. (Hint: it is critical that you have the right formula for $f_e$ and do not use a “constant” load per node value, can you explain why?)
* From everyday life steel material appears to be noticeable incompressible. Examine whether your implementation has volume (area in 2D) conservation (Hint: Explain the causes for your observations, consider carefully what could have “gone” wrong).

## Detailed Expected Learning Objectives in this Week

* Finite Element Method (FEM) Part 2 (Slides 15, 17, 18 and 19)
    * Work with concepts such as tensors when deriving equations (such as stress and strain fields).
    * Explain how principle of virtual work is related to weak form reformulations.
    * Apply FEM to a more complex problem such as linear elasticity.
    * Apply experimental validation to a FEM implementation of for instance linear elasticity.

In [1]:
%matplotlib widget

In [2]:
import igl
import meshplot as mp
import numpy as np
import matplotlib.pyplot as plt
import wildmeshing as wm

## Create the Computational Mesh
We will re-use the beam mesh generator from week 3

In [3]:
def make_beam_mesh(width, height, shape):
    x0 = -width/2.0
    y0 = -height/2.0
    I  = shape[0]
    J  = shape[1]
    dx = width/float(I)
    dy = height/float(J)
    V = np.zeros(((I+1)*(J+1),2),dtype=np.float64)
    for j in range(J+1):
        for i in range(I+1):
            k = i + j*(I+1)
            V[k,0] = x0 + i*dx
            V[k,1] = y0 + j*dy
    T = np.zeros((2*I*J,3),dtype=np.int32)
    for j in range(J):
        for i in range(I):
            k00 = (i  ) + (j  )*(I+1)
            k01 = (i+1) + (j  )*(I+1)
            k10 = (i  ) + (j+1)*(I+1)
            k11 = (i+1) + (j+1)*(I+1)
            e = 2*(i + j*I)
            if (i+j)%2:
                T[e,  :] = (k00,k01,k11)
                T[e+1,:] = (k00,k11,k10)
            else:
                T[e,  :] = (k10,k00,k01)
                T[e+1,:] = (k10,k01,k11)                    
    return V, T

V, T = make_beam_mesh(6.0,2.0,(12,6))


We will also reuse the triangle area computation function we used in week 4

In [4]:

fig = plt.figure()
plt.triplot(V[:,0],V[:,1],T)
plt.show()


def compute_triangle_areas(V,T):
    E = len(T) # Total number of triangles in the mesh
    A = np.zeros((E,),dtype=np.float64)
    for e in range(E):
        # Get triangle indices
        i = T[e,0]
        j = T[e,1]
        k = T[e,2]
        # Get triangle coordinates
        xi = V[i,0]
        xj = V[j,0]
        xk = V[k,0]
        yi = V[i,1]
        yj = V[j,1]
        yk = V[k,1]    
        
        dx1 = xk - xj
        dy1 = yk - yj
        dx2 = xi - xj
        dy2 = yi - yj

        A[e] =  (dx1*dy2 - dx2*dy1 ) / 2.0
    return A

A = compute_triangle_areas(V,T)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Linear elasticity FEM Theory Summary

The continous displacement field is given by interpolation of the discrete nodal displacement field

 $u(x) = N  U_e$
 
This is a 2-by-1 where 

$N = [ w_1 I, w_2 I, w_3 I ]$

This is 2-by-6 and

$U_e = [Ux(T(k,:));  Uy(T(k,:))]$

This is 6-by-one  and linear interpolation is used (bary-centric coordinates) 

   $w_1  =  area(x,2,3)  / A(k)$
   
   $w_2  =  area(3,1,x)  / A(k)$
   
   $w_3  =  area(x,1,2)  / A(k)$

The Cauchy strain tensor is given by

$\varepsilon(x) = S \, u(x) = S \, N \, U_e  = B \, U_e$

This is 3-by-1 where 

$B = S * N$

This is 3-by-6 and the differential operator is defined as

$S = \begin{bmatrix}dx & 0\\ 0 & dy\\ dy & dx\end{bmatrix}$
 
This is 3-by-2. The Cauchy Stress tensor can by found by the equation of state (EOS)
which defines a relation between stress and strain. For a neo-hookean
material model this yields the relation

$\sigma =  D \varepsilon$

This is 3-by-1 where

$D =  \frac{E}{1-\nu^2} \begin{bmatrix} 1 & \nu & 0\\
                     \nu &1 &0\\
                      0 & 0 & \frac{1-\nu}{2}\end{bmatrix}$

This is 3-by-3. Now by using the virtual work one ends up with a linear system for the unknown nodal displacements

$K_e U_e = F_e$

where

$K_e = B^T  D  B  A_e$

This is 6-by-6. To assemble the element-wise equations into one large simultaneous system one applies Newton's third law at the nodes.

In [5]:
N = len(V) # Total number of nodes in the mesh
E = len(T) # Total number of triangles in the mesh

Ke = np.zeros((6,E*6), dtype=np.float64)
K  = np.zeros( (N*2,N*2), dtype=np.float64)

# Compute the D (elasticity) matrix
#
# http://en.wikipedia.org/wiki/Young%27s_modulus
# http://en.wikipedia.org/wiki/Poisson%27s_ratio
#
# This is steel like parameters
Ey  = 69e9  # Young modulus
nu = 0.3    # Poisson ration

# TODO - Compute the D (elasticity) matrix
D  = np.array([
    [1,  nu, 0],
    [nu, 1,  0],
    [0,  0,  (1-nu) / 2]
]) * (Ey / (1 - nu ** 2))

# Now compute element stiffness matrices
for e in range(E):
    # Get triangle indices
    i = T[e,0]  
    j = T[e,1]  
    k = T[e,2]  
    # Get triangle coordinates
    xi = V[i,0]
    xj = V[j,0]
    xk = V[k,0]
    yi = V[i,1]
    yj = V[j,1]
    yk = V[k,1]
    
    # TODO - Compute spatial gradients of the barycentric coordinates
    Nix = -(yk - yj) / (2 * A[e])
    Njx = -(yi - yk) / (2 * A[e])
    Nkx = -(yj - yi) / (2 * A[e])
    
    Niy = (xk - xj) / (2 * A[e])
    Njy = (xi - xk) / (2 * A[e])
    Nky = (xj - xi) / (2 * A[e])
    
    # TODO - Compute the B matrix
    B = np.array([
        [Nix, 0,   Njx,   0, Nkx,   0],
        [0,   Niy,   0, Njy,   0, Nky],
        [Niy, Nix, Njy, Njx, Nky, Nkx]
    ])
    
    # TODO - Compute element stiffness matrix and store it in Ke array
    Ke[:, 6*e:6*e+6] = np.dot(np.dot(B.T, D), B) * A[e]
    

In [6]:
# Now do assembly process of global stiffness matrix
for e in range(E):
    # Get global triangle vertex indices
    i = T[e,0]  
    j = T[e,1]  
    k = T[e,2]    
    # Local order of vertex coordinates is i_x, i_y, j_x j_y, k_x, and  k_y. 
    # This is how local vertex indices (0,1,2,..,5) are mapped to global vertex
    # indices
    gidx = [ i, N + i, j,  N + j,  k, N + k]
    # Now we can add the element stiffness matrix to the global matrix using
    # our local-global vertex index mapping.
    
    ke = Ke[ :, (6*e):6*e+6 ]
    
    for m in range(6):
        for n in range(6):
            K[gidx[m], gidx[n]] = K[gidx[m], gidx[n]] + ke[m, n]
            
            

We will visualize the assembly process

In [7]:
fig = plt.figure()
ax = plt.subplot(111)
plt.spy(K);
ax.set_title('Fill pattern of K matrix');
ax.set_ylabel('Row index');
ax.set_xlabel('Column index');
plt.show()

fig = plt.figure()
ax = plt.subplot(111)
d, _ = np.linalg.eig(K)
plt.plot(np.sort( d ) );
ax.set_title('Eigenvalues of K matrix');
ax.set_xlabel('Eigenvalue Index')
ax.set_ylabel('Value');
fig.set_figheight(5)
plt.show()
print(np.sort( d ))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[-7.09271742e-05  1.33987946e-05  6.80976406e-05  5.90463195e+08
  2.26682988e+09  2.51500051e+09  5.91815427e+09  8.15644586e+09
  8.36057066e+09  9.71393525e+09  1.05708337e+10  1.21134080e+10
  1.23391248e+10  1.74370138e+10  1.84067755e+10  1.86566066e+10
  1.92897567e+10  2.10418809e+10  2.19105372e+10  2.21714177e+10
  2.47223045e+10  2.65191595e+10  2.73555895e+10  3.00426196e+10
  3.05913863e+10  3.23083492e+10  3.46604456e+10  3.55136935e+10
  3.55959173e+10  3.69913361e+10  4.06240182e+10  4.17645887e+10
  4.18590728e+10  4.54415782e+10  4.80049235e+10  4.86000873e+10
  4.88657138e+10  5.00657324e+10  5.14244482e+10  5.41439120e+10
  5.55224983e+10  5.65381994e+10  5.67747251e+10  5.72925047e+10
  5.96145873e+10  6.17556153e+10  6.27516310e+10  6.33923907e+10
  6.37918803e+10  6.48784650e+10  6.58291247e+10  6.62405285e+10
  6.90613207e+10  7.00413562e+10  7.10206536e+10  7.23921598e+10
  7.39845563e+10  7.43868085e+10  8.22269334e+10  8.27111219e+10
  8.45070834e+10  8.59805

Create an  external nodal force vector with a prescribed load at the right hand side of the bar.

In [8]:
load = -10e7/2
f = np.zeros((2*N,),dtype=np.float64)
indices = np.array(np.where(V[:,0]>2.9),dtype=np.int).flatten() + N
f[ indices ] =  load

Apply Direchlet boundary conditions to displacement field for the left vertices.

In [9]:
xindices = np.array(np.where( V[:,0] < -2.9), dtype=np.int).flatten()
yindices = xindices + N

indices = np.hstack((xindices, yindices))
values  = np.zeros(indices.shape,dtype=np.float64)

F = np.setdiff1d(np.arange(2*N), indices)

for i in range(len(indices)):
    index = indices[i]
    value = values[i]  
  
    # TODO - add boundary conditions here

After having applied the boundary conditions we can now solve for the displacement field

In [10]:
u = np.zeros(f.shape, dtype=np.float64)
u[indices] = values
KFF = K[F,:][:,F]
fF  = f[F]
u[F] = np.linalg.solve(KFF, fF)

From displacements we can compute the spatial deformed coordinates and visualize the deformed mesh.

In [11]:
x = V[:,0] + u[0:N]
y = V[:,1] + u[N:2*N]

fig = plt.figure()
plt.triplot(V[:,0],V[:,1],T)
plt.triplot(x,y,T)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

That is all folks

In [12]:
load = -10e7
f = np.zeros((2*N,),dtype=np.float64)
indices = np.array(np.where(V[:,1]<0.9),dtype=np.int).flatten() + N
f[ indices ] =  load

xindices = np.array(np.append(np.where(V[:,0] < -2.9), np.where( V[:,0] > 2.9)), dtype=np.int).flatten()
yindices = xindices + N

indices = np.hstack((xindices, yindices))
values  = np.zeros(indices.shape,dtype=np.float64)

F = np.setdiff1d(np.arange(2*N), indices)

for i in range(len(indices)):
    index = indices[i]
    value = values[i]  
    
u = np.zeros(f.shape, dtype=np.float64)
u[indices] = values
KFF = K[F,:][:,F]
fF  = f[F]
u[F] = np.linalg.solve(KFF, fF)
x = V[:,0] + u[0:N]
y = V[:,1] + u[N:2*N]

fig = plt.figure()
plt.triplot(V[:,0],V[:,1],T)
plt.triplot(x,y,T)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
load = +10e8
f = np.zeros((2*N,),dtype=np.float64)
indices = np.array(np.where(V[:,0]>0.9),dtype=np.int).flatten()
f[ indices ] =  load

xindices = np.array(np.where( V[:,0] < -2.9), dtype=np.int).flatten()
yindices = xindices + N

indices = np.hstack((xindices, yindices))
values  = np.zeros(indices.shape,dtype=np.float64)

F = np.setdiff1d(np.arange(2*N), indices)

for i in range(len(indices)):
    index = indices[i]
    value = values[i] 
    
u = np.zeros(f.shape, dtype=np.float64)
u[indices] = values

KFF = K[F,:][:,F]
fF  = f[F]
u[F] = np.linalg.solve(KFF, fF)
x = V[:,0] + u[0:N]
y = V[:,1] + u[N:2*N]

fig = plt.figure()
plt.triplot(V[:,0],V[:,1],T)
plt.triplot(x,y,T)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
def make_circle_mesh(radius, segments):
    K = segments
    X = np.array([ radius*np.cos(2*np.pi*k/K) for k in range(K)])
    Y = np.array([ radius*np.sin(2*np.pi*k/K) for k in range(K)])
    P = np.zeros((K,2))
    P[:,0] = X
    P[:,1] = Y
    L = np.array([(k,(k+1)%K) for k in range(K)],dtype=np.int32)
    
    V, T, _, _ = wm.triangulate_data(P, L, cut_outside=True)
    return (V, T)

V, T = make_circle_mesh(1, 36)
A = compute_triangle_areas(V,T)

N = len(V) # Total number of nodes in the mesh
E = len(T) # Total number of triangles in the mesh

Ke = np.zeros((6,E*6), dtype=np.float64)
K  = np.zeros( (N*2,N*2), dtype=np.float64)

# Compute the D (elasticity) matrix
#
# 

# http://en.wikipedia.org/wiki/Poisson%27s_ratio
#
# This is steel like parameters
Ey  = 69e9  # Young modulus
nu = 0.3    # Poisson ration

# TODO - Compute the D (elasticity) matrix
D  = np.array([
    [1,  nu, 0],
    [nu, 1,  0],
    [0,  0,  (1-nu) / 2]
]) * (Ey / (1 - nu ** 2))

# Now compute element stiffness matrices
for e in range(E):
    # Get triangle indices
    i = T[e,0]  
    j = T[e,1]  
    k = T[e,2]  
    # Get triangle coordinates
    xi = V[i,0]
    xj = V[j,0]
    xk = V[k,0]
    yi = V[i,1]
    yj = V[j,1]
    yk = V[k,1]
    
    # TODO - Compute spatial gradients of the barycentric coordinates
    Nix = -(yk - yj) / (2 * A[e])
    Njx = -(yi - yk) / (2 * A[e])
    Nkx = -(yj - yi) / (2 * A[e])
    
    Niy = (xk - xj) / (2 * A[e])
    Njy = (xi - xk) / (2 * A[e])
    Nky = (xj - xi) / (2 * A[e])
    
    # TODO - Compute the B matrix
    B = np.array([
        [Nix, 0,   Njx,   0, Nkx,   0],
        [0,   Niy,   0, Njy,   0, Nky],
        [Niy, Nix, Njy, Njx, Nky, Nkx]
    ])
    
    # TODO - Compute element stiffness matrix and store it in Ke array
    Ke[:, 6*e:6*e+6] = np.dot(np.dot(B.T, D), B) * A[e]
    
# Now do assembly process of global stiffness matrix
for e in range(E):
    # Get global triangle vertex indices
    i = T[e,0]  
    j = T[e,1]  
    k = T[e,2]    
    # Local order of vertex coordinates is i_x, i_y, j_x j_y, k_x, and  k_y. 
    # This is how local vertex indices (0,1,2,..,5) are mapped to global vertex
    # indices
    gidx = [ i, N + i, j,  N + j,  k, N + k]
    
    # Now we can add the element stiffness matrix to the global matrix using
    # our local-global vertex index mapping.
    ke = Ke[ :, (6*e):6*e+6 ]
    
    for m in range(6):
        for n in range(6):
            K[gidx[m], gidx[n]] = K[gidx[m], gidx[n]] + ke[m, n]
            
indices = np.array(np.where(V[:,1]>0.7),dtype=np.int).flatten() + N

def doit(load):
    f = np.zeros((2*N,),dtype=np.float64)
                
    indices = np.array(np.where(V[:,1]>0.7),dtype=np.int).flatten() + N

    f[ indices ] =  load

    xindices = np.array(np.where( V[:,1] < -0.7), dtype=np.int).flatten()
    yindices = xindices + N
    indices = np.hstack((xindices, yindices))
    values  = np.zeros(indices.shape,dtype=np.float64)

    F = np.setdiff1d(np.arange(2*N), indices)

    for i in range(len(indices)):
        index = indices[i]
        value = values[i]  

    u = np.zeros(f.shape, dtype=np.float64)
    u[indices] = values

    KFF = K[F,:][:,F]
    fF  = f[F]
    u[F] = np.linalg.solve(KFF, fF)
    
    return u

u = doit(-10*10 ** 8.999)

x = V[:,0] + u[0:N]
y = V[:,1] + u[N:2*N]

print(np.any(np.array(list(map(lambda x: x < 0,  y[indices - N]))).flatten()))

fig = plt.figure()
plt.triplot(V[:,0],V[:,1],T)
plt.triplot(x,y,T)
plt.show()

True


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …